## Concise Logistic Regression for Image Classification

- Shows a concise implementation of logistic regression for image classification
- Uses PyTorch

In [1]:
# 导入
import torch#安装torch库，支持在图形处理单元上计算张量
import torchvision#tochvision主要处理图像数据，包含一些常用的数据集、模型、转换函数等
import torch.nn as nn#nn包含了众多神经网络的常用模块
from torchvision import datasets, models, transforms#提供常用的数据预处理操作，主要包括对Tensor及PIL Image对象的操作
import os#导入os模块到当前程序
import numpy as np#用来处理数据或矩阵的库
import matplotlib.pyplot as plt
#当进行绘图时，或生成一个 figure 画布的时候，可以直接在你的python控制台里面生成图像
%matplotlib inline

# gpu空闲的话，使用它
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# download the data (uncomment if to download the data locally)
#!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
#!unzip hymenoptera_data.zip

In [2]:
# 创建数据加载器

data_dir = 'hymenoptera_data'#一个用于pytorch的小型数据集

# 自定义转换器来压平图像张量
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size#数组的列

    def __call__(self, img):
        result = torch.reshape(img, self.new_size)#将数组转换成一个img行new_size列的二维新数组
        return result

# 用于标准化和规范化数据集的转换
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),#调整容器的长度大小，使其能容纳224个元素
        transforms.CenterCrop(224),#裁剪图片，目标尺寸为224
        transforms.ToTensor(),#将Python中的数据类型转换为PyTorch张量，这个张量可以用来训练机器学习模型
        ReshapeTransform((-1,)) # 使数据变平
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),#调整容器的长度大小，使其能容纳224个元素
        transforms.CenterCrop(224),#裁剪图片，目标尺寸为224
        transforms.ToTensor(),#将Python中的数据类型转换为PyTorch张量，这个张量可以用来训练机器学习模型
        ReshapeTransform((-1,)) # 使数据变平
    ]),
}

# 加载相应的文件夹
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),#ImageFolder会将目录中的文件夹名自动转化成序列，每个文件夹下会存储相同的一个类别，文件夹名为类名
                                          data_transforms[x])#join()方法用于把数组中的所有元素放入一个字符串
                  for x in ['train', 'val']}

# 加载整个数据集；我们这里不使用minibatches
train_dataset = torch.utils.data.DataLoader(image_datasets['train'],#加载训练集
                                            batch_size=len(image_datasets['train']),#训练集长度
                                            shuffle=True)#shuffle=True用于打乱数据集,每次都会以不同的顺序返回

test_dataset = torch.utils.data.DataLoader(image_datasets['val'],#加载测试集
                                           batch_size=len(image_datasets['val']),#测试集长度
                                           shuffle=True)#shuffle=True用于打乱数据集,每次都会以不同的顺序返回

In [3]:
# 建立LR模型
class LR(nn.Module):
    def __init__(self, dim):
        super(LR, self).__init__()#子类把父类的__init__()放到自己的__init__()当中，这样子类就有父类的__init__()的东西进行初始化。
        self.linear = nn.Linear(dim, 1)#对输入的数据做线性变化
        nn.init.zeros_(self.linear.weight)#用标量0来填充weight
        nn.init.zeros_(self.linear.bias)#用标量0来填充bias

    def forward(self, x):
        x = self.linear(x)
        x = torch.sigmoid(x)#将x压缩至(0,1)范围之间
        return x 

In [4]:
# 预测函数
def predict(yhat, y):
    yhat = yhat.squeeze()#将yhat数组转换为秩为1的数组
    y = y.unsqueeze(0)#增加维度(0表示,在第一个位置增加维度)
    y_prediction = torch.zeros(y.size()[1])#生成和y第二行同样大小零矩阵
    for i in range(yhat.shape[0]):#i遍历yhat的行数
        if yhat[i] <= 0.5:#如果yhat数组第i个值小于等于0.5
            y_prediction[i] = 0#预测结果置为0
        else:#如果yhat数组第i个值大于0.5
            y_prediction[i] = 1#预测结果置为1
    return 100 - torch.mean(torch.abs(y_prediction - y)) * 100#根据公式计算准确率

In [5]:
# 模型配置
dim = train_dataset.dataset[0][0].shape[0]#dim取训练集第一个对象的行数

lrmodel = LR(dim).to(device)#将所有最开始读取数据时的tensor变量copy一份到device所指定的GPU上去，之后的运算都在GPU上进行
criterion = nn.BCELoss()#用来做二分类的损失函数
optimizer = torch.optim.SGD(lrmodel.parameters(), lr=0.0001)#构建优化器，parameters()获取lrmodel网络的参数，学习率为0.0001

In [8]:
# 训练模型
costs = []

for ITER in range(100):
    lrmodel.train()#训练模型
    #生成训练集的迭代器，next()函数将内部指针指向数组中的下一个元素,并输出
    x, y = next(iter(train_dataset))#取训练集中可迭代对象的元素
    test_x, test_y = next(iter(test_dataset))#取测试集中可迭代对象的元素

    # 向前
    yhat = lrmodel.forward(x.to(device))#正向传播，将模型参数放到gpu中

    cost = criterion(yhat.squeeze(), y.type(torch.FloatTensor).to(device))#测量预测值与实际值之间差异
    train_pred = predict(yhat, y)#对训练集进行预测

    # 向后
    optimizer.zero_grad()#将梯度清零
    cost.backward()#反向传播
    optimizer.step()#更新模型参数
    
    # 推理阶段
    lrmodel.eval()#评估模式
    #在评估模式下,batchNorm层,dropout层等用于优化训练而添加的网络层会被关闭,从而使得评估时不会发生偏移
    with torch.no_grad():#张量的计算过程中无需计算梯度
        yhat_test = lrmodel.forward(test_x.to(device))#反向传播测试集
        test_pred = predict(yhat_test, test_y)#对测试集进行预测

    if ITER % 10 == 0:#如果ITER是10的倍数
        costs.append(cost)#则将cost添加到损失集内

    if ITER % 10 == 0:#输出ITER为10的倍数时，损失、训练集准确率、测试集准确率
        print("Cost after iteration {}: {} | Train Acc: {} | Test Acc: {}".format(ITER, 
                                                                                    cost, 
                                                                                    train_pred,
                                                                                    test_pred))
   

Cost after iteration 0: 0.6931471228599548 | Train Acc: 50.40983581542969 | Test Acc: 45.75163269042969
Cost after iteration 10: 0.6691471338272095 | Train Acc: 64.3442611694336 | Test Acc: 54.24836730957031
Cost after iteration 20: 0.6513182520866394 | Train Acc: 68.44261932373047 | Test Acc: 54.24836730957031
Cost after iteration 30: 0.6367825269699097 | Train Acc: 68.03278350830078 | Test Acc: 54.24836730957031
Cost after iteration 40: 0.6245337128639221 | Train Acc: 69.67213439941406 | Test Acc: 54.90196228027344
Cost after iteration 50: 0.6139225363731384 | Train Acc: 70.90164184570312 | Test Acc: 56.20914840698242
Cost after iteration 60: 0.6045235395431519 | Train Acc: 72.54098510742188 | Test Acc: 56.86274337768555
Cost after iteration 70: 0.5960512161254883 | Train Acc: 74.18032836914062 | Test Acc: 57.51633834838867
Cost after iteration 80: 0.5883084535598755 | Train Acc: 73.77049255371094 | Test Acc: 57.51633834838867
Cost after iteration 90: 0.5811557769775391 | Train Acc: 

### References
- [A Logistic Regression Model from Scratch](https://colab.research.google.com/drive/1iBoJ0kngkOthy7SgVaVQA1aHEROt5mra?usp=sharing)